In [7]:
import sys
#sys.path.append('/Users/simon/git/lda/code')

from metabolomics import load_spectra
from keras.models import load_model
import numpy as np
import pandas as pd

from setup_paths import setup_paths
root_dir = 'G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/'
MIBIG_JSON_DIR,NODES_FILE,DEREP_ANNOTATIONS,MGF_FILE,EDGES_FILE,ROOT_PATH,ANTISMASH_DIR = setup_paths(root_dir)
FOLDERS = ['NRPS','Others','PKSI','PKS-NRP_Hybrids','PKSother','RiPPs','Saccharides','Terpene']
FOLDERS = ['NRPS']

MIBIG_JSON_DIR: G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/mibig_json
MGF_FILE: G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/spectra\METABOLOMICS-SNETS-2cdc2aa3-download_clustered_spectra-main.mgf
ANNOTATION_FILES: G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/spectra\derep.annotations.tsv
EDGES_FILE: G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/spectra\1eca77da22c84ae1aee490d0e4bc26f3.pairsinfo
NODES_FILE: G:/Dev/carnegie_nplinker_example/CarnegieDataForNPLinker/spectra\reformated.out


In [21]:
spec_fingerprint_model = load_model('../prediction_models/saved_spectrum_fingerprint_non_binary_model.h5')
fingerprint_family_model = load_model('../prediction_models/saved_fingerprint_families_non_binary_model.h5')

In [9]:
spectra = load_spectra(MGF_FILE)

Loaded 3125 molecules


In [10]:
print(spectra[0].peaks)

[(88.299004, 34.695), (89.059998, 67.724998), (133.085999, 131.283005), (190.518997, 1399.093018), (204.912994, 2232.0979), (213.080002, 2781.284912), (229.089005, 754.0), (241.074005, 926.260986), (284.070007, 1283.692017), (293.178009, 350.376007), (305.178009, 1886.754028), (307.194, 46.132)]


In [29]:
def load_master_file(spectra, number_of_bins = 1000):
    MAX_MASS = 1000
    BIN_SIZE = 1
    
    mol_ids = range(0, len(spectra))
    # Construct empty Pandas dataframe of correct size.
    # Number of rows is equal to the number of unique molecules (found in mol_ids).
    intensities = pd.DataFrame(0.0, index=mol_ids, columns=range(number_of_bins//BIN_SIZE), dtype=float)
    
    # Populate the dataframe using each molecule's filename to place data in the correct row.
    for index, spectrum in enumerate(spectra):
        peaks = spectrum.peaks
        for peak in peaks:
            if peak[0] <= 1000:
                intensities.at[index, round(float(peak[0]))-1] = float(peak[1])
    
    # Convert populated dataframe into a numpy array for use by neural networks.
    np_matrix = intensities.values
    return np_matrix

binned_spectra = load_master_file(spectra)

In [32]:
normalised_spectra = np.log(binned_spectra+1)
print(normalised_spectra)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
predicted_fingerprint = spec_fingerprint_model.predict(normalised_spectra)
predicted_families = fingerprint_family_model.predict(predicted_fingerprint)

In [35]:
print(predicted_families)

[[0.03109328 0.07207853 0.01606273 ... 0.00970557 0.49074814 0.23700576]
 [0.08192425 0.18247652 0.0281545  ... 0.00572589 0.21462008 0.17954497]
 [0.05943411 0.23582199 0.05601655 ... 0.00229134 0.32919785 0.11122002]
 ...
 [0.03424589 0.17623684 0.02856023 ... 0.0085697  0.34152547 0.15714952]
 [0.01819078 0.08360698 0.01617763 ... 0.01581099 0.52307755 0.18000518]
 [0.01109693 0.24820817 0.0160438  ... 0.00854864 0.472931   0.11307999]]


2.2.4
1.12.0
